In [ ]:
cd ../

In [ ]:
from models.lit_models import BaseModel

In [ ]:
import torch
import torchvision.transforms as T

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
checkpoint = "artifacts/weights/fasterrcnn-1kimages-weightdecay.ckpt"

In [ ]:
optimizer = torch.optim.AdamW
weight_decay = 0.0
learning_rate = 6.918309709189363e-05
model = BaseModel.load_from_checkpoint(checkpoint_path=checkpoint,
                                       optimizer=optimizer,
                                       learning_rate=learning_rate,
                                       weight_decay=weight_decay,
                                       pretrained_backbone=True,
                                       num_classes=2,
                                       )
model.to(device)
model.eval()
print("Model in eval")

In [ ]:
from PIL import Image, ImageDraw, ImageFont

In [ ]:
font = ImageFont.truetype("notebooks/Arial Unicode.ttf", 40)

In [ ]:
test_img = Image.open("test2.jpeg").convert("L")
w, h = test_img.size
draw = ImageDraw.Draw(test_img)
test_img_small = test_img.resize((212, 300))
test_img_tensor = T.ToTensor()(test_img_small)
test_img_tensor = test_img_tensor.repeat(3, 1, 1)
test_img_tensor = test_img_tensor.to(device)

In [ ]:
losses, detections = model([test_img_tensor])

In [ ]:
for idx, box in enumerate(detections[0]['boxes']):
    x1 = (box[0]/212)*w
    x2 = (box[2]/212)*w
    y1 = (box[1]/300)*h
    y2 = (box[3]/300)*h
    nbox = [x1, y1, x2, y2]
    score = detections[0]['scores'][idx].detach().item()
    if score > 0.5:
        draw.rectangle(nbox, outline="red", width=10)
        draw.text([x1,y1-20], str(round(score)), font=font)

In [ ]:
test_img